<a href="https://colab.research.google.com/github/krishang-parakh/linkedin-job-trends/blob/main/Job_Market_Trend_Analysis_Using_LinkedIn_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pyspark
!pip install graphframes
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LinkedIn Job Postings").master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.1-spark3.0-s_2.12").getOrCreate()
from pyspark.ml.feature import Imputer

In [8]:
file_paths = {
    "benefits": "benefits.csv",
    "companies": "companies.csv",
    "company_industries": "company_industries.csv",
    "company_specialities": "company_specialities.csv",
    "employee_counts": "employee_counts.csv",
    "industries": "industries.csv",
    "job_industries": "job_industries.csv",
    "job_postings": "job_postings.csv",
    "job_skills": "job_skills.csv",
    "salaries": "salaries.csv",
    "skills": "skills.csv",
}

In [9]:
# Assuming simplified schema for demonstration, adjust according to your CSV files
job_postings_schema = "job_id STRING, title STRING, description STRING, location STRING, company_id STRING"
job_industries_schema = "job_id STRING, industry_id STRING"
job_skills_schema = "job_id STRING, skill_abr STRING"
industries_schema = "industry_id STRING, industry_name STRING"
skills_schema = "skill_abr STRING, skill_name STRING"
salaries_schema = "job_id STRING, salary INT, currency STRING"
employee_counts_schema = "company_id STRING, employee_count INT"

# Read CSV files into DataFrames, assuming header=True and inferring schema for demonstration
job_postings_df = spark.read.option("header", "true").csv("job_postings.csv", inferSchema=True)
job_industries_df = spark.read.option("header", "true").csv("job_industries.csv", inferSchema=True)
job_skills_df = spark.read.option("header", "true").csv("job_skills.csv", inferSchema=True)
industries_df = spark.read.option("header", "true").csv("industries.csv", inferSchema=True)
skills_df = spark.read.option("header", "true").csv("skills.csv", inferSchema=True)

# Perform joins
joined_df = job_postings_df \
    .join(job_industries_df, "job_id") \
    .join(industries_df, "industry_id") \
    .join(job_skills_df, "job_id") \
    .join(skills_df, "skill_abr")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/job_postings.csv.

In [ ]:
import shutil
import time

streaming_directory = "stream_simulation/"
streaming_checkpoint_directory = "stream_checkpoint/"

# Convert the DataFrame to JSON and save it to simulate the initial load
joined_df.write.mode("overwrite").json(streaming_directory)

# Assuming the schema of joined_df is correctly defined and does not include duplicate columns
stream_df = spark.readStream.schema(joined_df.schema).json(streaming_directory)

# Start the streaming query to output to a memory table
query = stream_df.writeStream.queryName("streaming_data").format("memory").outputMode("update").start()

# Function to perform SQL queries and display results
def display_streaming_data():
    # Display top industries by job postings
    print("Job Postings by Industry:\n")
    spark.sql("""
        SELECT industry_name, COUNT(*) AS job_count
        FROM streaming_data
        GROUP BY industry_name
        ORDER BY job_count DESC
    """).show(truncate=False)

    # Display top skills by job postings
    print("Job Availability by Skill Type:\n")
    spark.sql("""
        SELECT skill_name, COUNT(*) AS job_count
        FROM streaming_data
        GROUP BY skill_name
        ORDER BY job_count DESC
    """).show(truncate=False)

    # Select skill names along with the list of industries that require them
    print("List of Industries by Skills Required:\n")
    spark.sql("""
        SELECT job_skills.skill_name, collect_list(distinct job_industries.industry_name) FROM job_skills JOIN job_industries
        ON job_skills.job_id = job_industries.job_id
        GROUP BY
          job_skills.skill_name
    """).show(truncate=False)

# Simulate streaming by appending new data
for i in range(1, 16):
    print(f"Iteration: {i}")
    # Specify the source file for the initial data
    source_file = streaming_directory + "part-00000"  # This assumes only one part file exists; adjust as necessary
    destination_file = streaming_directory + f"data_{i}.json"

    time.sleep(5)  # Adjust based on your system
    display_streaming_data()

# Stop the query
query.stop()

In [ ]:
for name, path in file_paths.items():
    df = spark.read.csv(path, header=True, inferSchema=True)
    rdd = df.rdd
    columns = df.columns  # Extract column names
    count = rdd.count()  # Get the number of observations in the RDD

    # Print dataset name, number of observations, and columns
    print(f"Dataset Name: {name}")
    print(f"Number of Observations: {count}")
    print("Columns:", columns)

    # For each column, print the schema description (data type)
    # Since RDDs do not have schema, we extract schema from DataFrame
    for field in df.schema.fields:
        print(f"{field.name}: {field.dataType.simpleString()}")

    print("--------------------------------------------------")

In [ ]:
from pyspark.sql.functions import *

# Assuming job_postings_df is already defined as per your schema
job_postings_df = spark.read.csv("job_postings.csv", header=True, schema=job_postings_schema)

# Utilize DataFrame API to count nulls directly without converting to RDD
null_counts = job_postings_df.select([count(when(col(c).isNull(), c)).alias(c) for c in job_postings_df.columns])

print("Count of Null Values in Each Field:")
null_counts.show()

In [ ]:
# Assuming you're back to static DataFrames for RDD operations
# Let's take job_postings_df as an example
job_postings_df = spark.read.csv("job_postings.csv", schema=job_postings_schema, header=True)

# Convert to RDD for manipulation
job_postings_rdd = job_postings_df.rdd

# Column descriptions
print("Column Descriptions:")
print(job_postings_df.dtypes)

# Initial few rows
print("Initial Rows:")
for row in job_postings_rdd.take(5):
    print(row)

print("--------------------------------------------------")

In [ ]:
structured_conditions = """
For companies in the same city:
- "Rivals": Both have a size >= 5.
- "Partners": One has a size >= 5 and the other has a size between 3 and 5.
- "Unrelated": One has a size >= 5 and the other has a size < 3.
- "Mergers": Both have a size < 5 but >= 3.
- "Allies": One has a size >= 3 but < 5, and the other has a size < 3.
- "Sisters": Both have a size < 3.

For companies in different cities:
- "Allies": If only one has a size >= 4.
- "Partners": If both have a size >= 4.
- "Sisters": If both have a size <= 3.
- "Rivals": For all other conditions.
"""

# Output the structured conditions
print(structured_conditions)

In [ ]:
nodes_df = spark.read.csv("Nodes.csv", header=True, inferSchema=True)

# Self-join the DataFrame to create all possible pairs of companies
edges_df = nodes_df.alias("src").join(nodes_df.alias("dst"), col("src.city") == col("dst.city"), "inner") \
    .select(
        col("src.id").alias("src"),
        col("dst.id").alias("dst"),
        when(
            (col("src.size") >= 5) & (col("dst.size") >= 5), "rivals"
        ).when(
            (col("src.size") >= 5) & (col("dst.size") < 5) & (col("dst.size") >= 3), "partners"
        ).when(
            (col("src.size") >= 5) & (col("dst.size") < 3), "unrelated"
        ).when(
            (col("src.size") < 5) & (col("src.size") >= 3) & (col("dst.size") < 5) & (col("dst.size") >= 3), "mergers"
        ).when(
            (col("src.size") >= 3) & (col("src.size") < 5) & (col("dst.size") < 3), "allies"
        ).when(
            (col("src.size") < 3) & (col("dst.size") < 3), "sisters"
        ).otherwise("rivals").alias("relation")
    )

# Filter out duplicate pairs and self-relationships
edges_df = edges_df.filter("src != dst").distinct()

# Display the edges DataFrame
edges_df.show(truncate=False)

# For different cities, we have to create additional rules and append to the current edges DataFrame
different_city_edges_df = nodes_df.alias("src").join(nodes_df.alias("dst"), col("src.city") != col("dst.city"), "inner") \
    .select(
        col("src.id").alias("src"),
        col("dst.id").alias("dst"),
        when(
            (col("src.size") >= 4) & (col("dst.size") >= 4), "partners"
        ).when(
            (col("src.size") <= 3) & (col("dst.size") <= 3), "sisters"
        ).otherwise("rivals").alias("relation")
    )

# Filter out duplicate pairs and self-relationships
different_city_edges_df = different_city_edges_df.filter("src != dst").distinct()

# Union the two datasets
final_edges_df = edges_df.union(different_city_edges_df).distinct()

# Write the edges DataFrame to a CSV file
final_edges_df.write.csv("Edges.csv", header=True, mode="overwrite")

In [ ]:
import networkx as nx
from graphframes import GraphFrame
import matplotlib.pyplot as plt

vertices = spark.read.option('header', 'true').csv('Nodes.csv')
edges = spark.read.option('header', 'true').csv('Edges.csv')

vertices.show(5)
edges.show(5)

mygraph = GraphFrame(vertices, edges)

mygraph.vertices.show()
mygraph.edges.show()

# result = mygraph.filterEdges("relationship = 'requires_skill'").filterVertices("size = 7")
# result.vertices.show()
# result.edges.show()

def plot_undirected_graph(edge_list):
    plt.figure(figsize=(9, 9))
    gplot = nx.Graph()
    for row in edge_list.select("src", "dst").take(100):
        gplot.add_edge(row["src"], row["dst"])
    nx.draw(gplot, with_labels=True, font_weight="bold", node_size=500)
    plt.show()

In [ ]:
plot_undirected_graph(mygraph.edges)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

job_postings_df = spark.read.csv("job_postings.csv", header=True, inferSchema=True)
salaries_df = spark.read.csv("salaries.csv", header=True, inferSchema=True)

data_df = job_postings_df.join(salaries_df, "job_id")

In [ ]:
feature_cols = job_postings_df.columns  # Replace with actual feature columns
feature_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Prepare the data with features and label
data_df = feature_assembler.transform(data_df).select(col("salary").alias("label"), "features")

# Split the data into training and test sets
train_df, test_df = data_df.randomSplit([0.7, 0.3])

In [ ]:
# Initialize models
lr = LinearRegression()
dt = DecisionTreeRegressor()
rf = RandomForestRegressor()

# Train the models
lr_model = lr.fit(train_df)
dt_model = dt.fit(train_df)
rf_model = rf.fit(train_df)

# Make predictions
lr_predictions = lr_model.transform(test_df)
dt_predictions = dt_model.transform(test_df)
rf_predictions = rf_model.transform(test_df)

# Evaluate the models
evaluator = RegressionEvaluator()

lr_rmse = evaluator.evaluate(lr_predictions)
dt_rmse = evaluator.evaluate(dt_predictions)
rf_rmse = evaluator.evaluate(rf_predictions)

# Print RMSE values
print(f"Linear Regression RMSE: {lr_rmse}")
print(f"Decision Tree RMSE: {dt_rmse}")
print(f"Random Forest RMSE: {rf_rmse}")

# Determine the best model based on RMSE
best_model = min([(lr_model, lr_rmse), (dt_model, dt_rmse), (rf_model, rf_rmse)], key=lambda x: x[1])
print(f"The best model is: {best_model[0].__class__.__name__} with a RMSE of: {best_model[1]}")